In [201]:
### Utility Functions
from IPython.display import display, HTML
import pandas as pd
import sqlite3
from sqlite3 import Error

def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql, drop_table_name=None):
    
    if drop_table_name: # You can optionally pass drop_table_name to drop the table. 
        try:
            c = conn.cursor()
            c.execute("""DROP TABLE IF EXISTS %s""" % (drop_table_name))
        except Error as e:
            print(e)
    
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows


In [202]:
def create_crime_table(data_filename, normalized_database_filename):

    conn = create_connection('crime.db')
       
    create_table_crime_sql = """ CREATE TABLE [Crime] (
    [County] TEXT NOT NULL PRIMARY KEY,
    [Population_2020] TEXT NOT NULL,
    [Index_Crime_Count] TEXT NOT NULL,
    [Index_Crime_Rate] TEXT NOT NULL,
    [Violent_Crime_Count] TEXT NOT NULL,
    [Violent_Crime_Rate] TEXT NOT NULL,
    [Property_Crime_Count] TEXT NOT NULL,
    [Property_Crime_Rate] TEXT NOT NULL, 
    [Violent_Crime_With_Firearm_Count] TEXT NOT NULL,
    [Violent_Crime_With_Firearm_Rate] TEXT NOT NULL
    ); """
    
    def insert_crime(conn, values):
        sql = ''' INSERT INTO Crime(County
        ,Population_2020
        ,Index_Crime_Count
        ,Index_Crime_Rate
        ,Violent_Crime_Count
        ,Violent_Crime_Rate
        ,Property_Crime_Count
        ,Property_Crime_Rate
        ,Violent_Crime_With_Firearm_Count
        ,Violent_Crime_With_Firearm_Rate)
        VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?) '''
        cur = conn.cursor()
        cur.executemany(sql, values)
        return cur.lastrowid    

    with open(data_filename, 'r') as f:
        crime_lst = []
        for line in f:
            if line.split('\t') not in crime_lst:
                crime_lst.append(line.rstrip('\n').split('\t'))
        crime_lst=sorted(crime_lst[1:])
        
    with conn:
        lst_tup_reg=[]
        lst=[]
        create_table(conn, create_table_crime_sql,'crime')
        for i in range(len(crime_lst)):
            lst_tup_reg.append(tuple(''.join(crime_lst[i]).split(',')))
        insert_crime(conn,lst_tup_reg)

In [203]:
data_filename = '2020-county-index-rates.csv'
normalized_database_filename = 'crime.db'
create_crime_table(data_filename, normalized_database_filename)
conn = create_connection(normalized_database_filename)
cur = conn.cursor()
df = pd.read_sql_query("""SELECT * FROM Crime""", conn)
print(df)
conn.close()

         County Population_2020 Index_Crime_Count Index_Crime_Rate  \
0        Albany       303723.00           7412.00          2440.40   
1      Allegany        45491.00            401.00           881.50   
2         Bronx      1413534.00          36217.00          2562.20   
3        Broome       188206.00           5180.00          2752.30   
4   Cattaraugus        73437.00            868.00          1182.00   
..          ...             ...               ...              ...   
57   Washington        60583.00            350.00           577.70   
58        Wayne        88933.00           1332.00          1497.80   
59  Westchester       963247.00          10306.00          1069.90   
60      Wyoming        39363.00            304.00           772.30   
61        Yates        24704.00            221.00           894.60   

   Violent_Crime_Count Violent_Crime_Rate Property_Crime_Count  \
0              1115.00             367.10              6297.00   
1                73.00     